## 0. Import data

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import os
os.chdir("/Users/yaroslavboiko/Desktop/HW_Data_Analytics_1")

data = pd.read_stata('sipp1991.dta').values

## 1. Preparing the subset

In [2]:
import numpy as np
data=np.delete(data,[0,2,11],1)
y_var=data[:,0]
d_var=data[:,8]
x_var=np.delete(data,[0,8],1)

## 2. Run regression 

In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


reg = LinearRegression().fit(x_var, y_var)
reg.score(x_var, y_var)

0.22944349768021954

## 3. Prediction with advanced algorithm

In [4]:
import pandas as pd
import numpy as np

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_var, y_var, test_size=0.2, random_state=0)

In [6]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [8]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 21762.665683308118
Mean Squared Error: 3812939766.465818
Root Mean Squared Error: 61749.00619820385


## 4. Using Double ML

In [9]:
import numpy as np
import warnings
import math
from scipy.optimize import minimize
import sklearn
from sklearn import tree
from sklearn import linear_model
from sklearn import ensemble
from sklearn import neural_network
from sklearn.model_selection import KFold
import statsmodels.api as sm
import warnings
import time

In [10]:
class ML2Estimator:

    def __init__(self, method="Tree", method_binary=None, method_options=None, method_options_binary=None):
        self.method=method
        self.method_options=method_options
        if method_binary is None:
            if method=="Lasso":
                self.method_binary="Lasso Logit"
            elif method=="Ridge":
                self.method_binary="Ridge Logit"
            else:
                self.method_binary=self.method
        else:
            self.method_binary=method_binary
        self.method_options_binary=method_options_binary
        if method_options_binary is None:
            if self.method==self.method_binary:
                self.method_options_binary=self.method_options
        self.method_class=self._define_model(binary_outcome=False)
        self.method_class_binary=self._define_model(binary_outcome=True)
        self.pl_beta=None
        self.pl_se=None
        self.interactive_beta=None
        self.interactive_se=None

    def _define_lasso(self,binary_outcome):
        if binary_outcome:
            m_options=self.method_options_binary
        else:
            m_options=self.method_options
        lasso_options={"eps":0.001, "cv":10,"n_alphas":100,"alphas":None,"fit_intercept":True, "precompute":'auto',
            "max_iter":5000, "tol":0.0001, "copy_X":True, "verbose":False, "n_jobs":1, "positive":False, "random_state":None,
            "selection":'cyclic'}
        if m_options!=None:
            if isinstance(m_options,dict):
                for i in m_options:
                    if i in lasso_options:
                        lasso_options[i]=m_options[i]
                    else:
                        print("Error: ", i," is not a valid option entry")
                        print("valid option entries include:")
                        for i in lasso_options:
                            print(i)
                        raise NameError("Invalid option entry")		
            else:
                raise NameError("options must take the form of a dictionary")
        #print (lasso_options)
        eps=lasso_options["eps"]
        cv=lasso_options["cv"]
        n_alphas=lasso_options["n_alphas"]
        alphas=lasso_options["alphas"]
        fit_intercept=lasso_options["fit_intercept"]
        precompute=lasso_options["precompute"]
        max_iter=lasso_options["max_iter"]
        tol=lasso_options["tol"]
        copy_X=lasso_options["copy_X"]
        verbose=lasso_options["verbose"]
        n_jobs=lasso_options["n_jobs"]
        positive=lasso_options["positive"]
        random_state=lasso_options["random_state"]
        selection=lasso_options["selection"]
        return linear_model.LassoCV(cv=cv, eps=eps, fit_intercept=fit_intercept, n_alphas=n_alphas, alphas=alphas,
                precompute=precompute,max_iter=max_iter,tol=tol,copy_X=copy_X,verbose=verbose, n_jobs=n_jobs, positive=positive,
                random_state=random_state,selection=selection)

    def _define_lasso_logit(self,binary_outcome):
        ll_options={"cv": 10, "Cs": 10, "solver":'SLSQP', "solver_options":None,"low_val": 1E-3, "high_val":1E3}
        m_options=self.method_options_binary
        if m_options!=None:
            if isinstance(m_options,dict):
                for i in m_options:
                    if i in ll_options:
                        ll_options[i]=m_options[i]
                    else:
                        print("Error: ",i," is not a valid option entry")
                        print("valid option entries include:")
                        for i in ll_options:
                            print(i)
                        raise NameError("Invalid option entry")	
                        raise NameError("Invalid option entry")		
            else:
                raise NameError("options must take the form of a dictionary")
        #print (ll_options)
        cv=ll_options["cv"]
        Cs=ll_options["Cs"]
        solver=ll_options["solver"]
        solver_options=ll_options["solver_options"]
        low_val=ll_options["low_val"]
        high_val=ll_options["high_val"]
        return LassoLogitCV(cv=cv, Cs=Cs, solver=solver, low_val=low_val, high_val=high_val)

    def _define_random_forest(self,binary_outcome):
        if binary_outcome:
            m_options=self.method_options_binary
        else:
            m_options=self.method_options
        rf_options={"n_estimators": 1000, "criterion": 'mse', "max_depth": None, "min_samples_split": 5, "min_samples_leaf": 5, 
            "min_weight_fraction_leaf":0.0, "max_features":"auto", "max_leaf_nodes":None, "min_impurity_decrease":1e-7, 
            "bootstrap":True, "oob_score":False, "n_jobs":1, "random_state":None, "verbose":0, "warm_start":False}
        if m_options!=None:
            if isinstance(m_options,dict):
                for i in m_options:
                    if i in rf_options:
                        rf_options[i]=m_options[i]
                    else:
                        print("Error: ",i," is not a valid option entry")
                        print("valid option entries include:")
                        for i in rf_options:
                            print(i)
                        raise NameError("Invalid option entry")	
                        raise NameError("Invalid option entry")		
            else:
                raise NameError("options must take the form of a dictionary")
        #print (rf_options)
        n_estimators=rf_options["n_estimators"]
        criterion=rf_options["criterion"]
        max_depth=rf_options["max_depth"]
        min_samples_split=rf_options["min_samples_split"]
        min_samples_leaf=rf_options["min_samples_leaf"]
        min_weight_fraction_leaf=rf_options["min_weight_fraction_leaf"]
        max_features=rf_options["max_features"]
        max_leaf_nodes=rf_options["max_leaf_nodes"]
        min_impurity_decrease=rf_options["min_impurity_decrease"]
        bootstrap=rf_options["bootstrap"]
        oob_score=rf_options["oob_score"]
        n_jobs=rf_options["n_jobs"]
        random_state=rf_options["random_state"]
        verbose=rf_options["verbose"]
        warm_start=rf_options["warm_start"]
        return ensemble.RandomForestRegressor(n_estimators=n_estimators,criterion=criterion,max_depth=max_depth,
            min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, min_weight_fraction_leaf=min_weight_fraction_leaf,
            max_features=max_features, max_leaf_nodes=max_leaf_nodes, min_impurity_decrease=min_impurity_decrease, bootstrap=bootstrap,
            oob_score=oob_score,n_jobs=n_jobs, random_state=random_state, verbose=verbose)

    def _define_regression_tree(self,binary_outcome):
        if binary_outcome:
            m_options=self.method_options_binary
        else:
            m_options=self.method_options
        tree_options={"criterion":'mse', "splitter":'best', "max_depth":None, "min_samples_split":2, "min_samples_leaf":1,
            "min_weight_fraction_leaf":0.0, "max_features":None, "random_state":None, "max_leaf_nodes":None, 
            "min_impurity_decrease":1e-07, "presort":False, "n_jobs": 1, "cv": 10, "search_range_low": 1, "search_range_high":11}
        if m_options!=None:
            if isinstance(m_options,dict):
                for i in m_options:
                    if i in tree_options:
                        tree_options[i]=m_options[i]
                    else:
                        print("Error: ",i," is not a valid option entry")
                        print("valid option entries include:")
                        for i in tree_options:
                            print(i)
                        raise NameError("Invalid option entry")	
                        raise NameError("Invalid option entry")		
            else:
                raise NameError("options must take the form of a dictionary")
        #print (tree_options)
        criterion=tree_options["criterion"]
        splitter=tree_options["splitter"]
        min_samples_split=tree_options["min_samples_split"]
        min_weight_fraction_leaf=tree_options["min_weight_fraction_leaf"]
        max_features=tree_options["max_features"]
        random_state=tree_options["random_state"]
        max_leaf_nodes=tree_options["max_leaf_nodes"]
        min_impurity_decrease=tree_options["min_impurity_decrease"]
        presort=tree_options["presort"]
        n_jobs=tree_options["n_jobs"]
        cv=tree_options["cv"]
        search_range=range(tree_options["search_range_low"],tree_options["search_range_high"])
        parameters = {'max_depth':search_range}
        clf = sklearn.model_selection.GridSearchCV(tree.DecisionTreeRegressor(criterion=criterion, splitter=splitter,
                min_samples_split=min_samples_split, min_weight_fraction_leaf=min_weight_fraction_leaf, max_features=max_features,
                random_state=random_state,max_leaf_nodes=max_leaf_nodes, presort=presort),
                parameters, n_jobs=n_jobs,cv=cv)
        return clf
    def _define_regression_tree_boosted(self,binary_outcome):
        if binary_outcome:
            m_options=self.method_options_binary
        else:
            m_options=self.method_options
        ada_options={"base_estimator":None, "n_estimators":100, "learning_rate":0.001, "loss":'exponential', "random_state":None}
        if m_options!=None:
            if isinstance(m_options,dict):
                for i in m_options:
                    if i in ada_options:
                        ada_options[i]=m_options[i]
                    else:
                        print("Error: ",i," is not a valid option entry")
                        print("valid option entries include:")
                        for i in ada_options:
                            print(i)
                        raise NameError("Invalid option entry")	
                        raise NameError("Invalid option entry")		
            else:
                raise NameError("options must take the form of a dictionary")
        #print (ada_options)
        base_estimator=ada_options["base_estimator"]
        n_estimators=ada_options["n_estimators"]
        learning_rate=ada_options["learning_rate"]
        loss=ada_options["loss"]
        random_state=ada_options["random_state"]
        return sklearn.ensemble.AdaBoostRegressor(base_estimator=base_estimator, n_estimators=n_estimators, learning_rate=learning_rate,
            loss=loss, random_state=random_state)
    def _define_ridge(self,binary_outcome):
        if binary_outcome:
            m_options=self.method_options_binary
        else:
            m_options=self.method_options
        ridge_options={"alphas":40, "fit_intercept":True, "scoring":None,
            "cv":10, "gcv_mode":None, "store_cv_values":False}
        if m_options!=None:
            if isinstance(m_options,dict):
                for i in m_options:
                    if i in ridge_options:
                        ridge_options[i]=m_options[i]
                    else:
                        print("Error: ",i," is not a valid option entry")
                        print("valid option entries include:")
                        for i in ridge_options:
                            print(i)
                        raise NameError("Invalid option entry")	
                        raise NameError("Invalid option entry")		
            else:
                raise NameError("options must take the form of a dictionary")
        #print (ridge_options)
        alphas=ridge_options["alphas"]
        if (isinstance(alphas,int)):
            alphas=np.exp(np.linspace(-3*math.log(10),3*math.log(10),alphas))
        fit_intercept=ridge_options["fit_intercept"]
        scoring=ridge_options["scoring"]
        cv=ridge_options["cv"]
        gcv_mode=ridge_options["gcv_mode"]
        store_cv_values=ridge_options["store_cv_values"]
        return linear_model.RidgeCV(alphas=alphas, fit_intercept=fit_intercept,
                scoring=scoring, cv=cv, gcv_mode=gcv_mode, store_cv_values=store_cv_values)

    def _define_ridge_logit(self,binary_outcome):
        rl_options={"cv": 10, "Cs": 10, "solver":'SLSQP', "solver_options":None,"low_val": 1E-3, "high_val":1E3}
        m_options=self.method_options_binary
        if m_options!=None:
            if isinstance(m_options,dict):
                for i in m_options:
                    if i in rl_options:
                        rl_options[i]=m_options[i]
                    else:
                        print("Error: ",i," is not a valid option entry")
                        print("valid option entries include:")
                        for i in rl_options:
                            print(i)
                        raise NameError("Invalid option entry")	
                        raise NameError("Invalid option entry")		
            else:
                raise NameError("options must take the form of a dictionary")
        #print (rl_options)
        cv=rl_options["cv"]
        Cs=rl_options["Cs"]
        solver=rl_options["solver"]
        solver_options=rl_options["solver_options"]
        low_val=rl_options["low_val"]
        high_val=rl_options["high_val"]
        return RidgeLogitCV(cv=cv, Cs=Cs, solver=solver, low_val=low_val, high_val=high_val)
    def _define_model(self,binary_outcome=False):
        if binary_outcome:
            local_method=self.method_binary
        else:
            local_method=self.method
        if local_method=="Tree":
            return self._define_regression_tree(binary_outcome)
        if local_method=="Random Forest":
            return self._define_random_forest(binary_outcome)
        if local_method=="Boosted Tree":
            return self._define_regression_tree_boosted(binary_outcome)
        if local_method=="Ridge":
            return self._define_ridge(binary_outcome)
        if local_method=="Ridge Logit":
            if binary_outcome:
                return self._define_ridge_logit(binary_outcome)
            else:
                raise NameError("The method 'Ridge Logit' can only be used to predict the binary outcome" )
        if local_method=="Lasso":
            return self._define_lasso(binary_outcome)
        if local_method=="Lasso Logit":
            if binary_outcome:
                return self._define_lasso_logit(binary_outcome)
            else:
                raise NameError("The method 'Lasso Logit' can only be used to predict the binary outcome" )
        else:
            raise NameError("You have entered an unrecognized machine learning mehod.\nRecognized methods include: `Lasso', `Ridge', `Tree' `Random Forest', and `Boosted Tree'")	

    def fit(self,X,Y,binary_outcome):
        """
        The 'fit' method returns a fitted model of the regressors X on the outcome Y. If binary_outcome is True, then
        the fitted model corresponds to self.method_class_binary, with self.method_options_binary. If binary_outcome is False, then
        the fitted model corresponds to self.method_class, with self.method_options.
        Parameters
        ----------
        X is a mxn numpy array where m is the number of observations and n is the number of regressors.
        Y is a row vector of length m.
        """
        if binary_outcome:
            if self.method_binary=="Tree":
                return self.method_class_binary.fit(X=X,y=Y).best_estimator_
            else:
                return self.method_class_binary.fit(X=X,y=Y)
        else:
            if self.method_binary=="Tree":
                return self.method_class.fit(X=X,y=Y).best_estimator_
            else:
                return self.method_class.fit(X=X,y=Y)
    def _find_residuals(self, y_use,y_out,x_use,x_out,binary_outcome=False):
        """
        The 'find_residuals' method uses the appropriate ml method (based on binary_outcomes value), and fits
        the ml method using the regressors x_use nad the outcome y_use. It then uses this same model to predict
        Parameters
        ----------
        y_out given x_out, and returns the predicted values of y_out and the residuals of the predicted values
        x_use is an mxn numpy array where m is the number of observations and n is the number of regressors.
        y_use is a row vector of length m. The same idea applies to x_out and y_out.
        """
        model=self.fit(x_use,y_use,binary_outcome=binary_outcome)
        yhat_out=model.predict(x_out)
        res_out=y_out-yhat_out
        return yhat_out, res_out
    def pl_estimate(self,X,y,d,test_size=.5, normalize=True,second_order_terms=False, verbose=True, standard_errors="Mackinnon"):
        """
        The pl_estimate method is an implementation of the partial linear estimation procedure developed 
        in `Double Machine  Learning for Treatment and Causal Parameters' by Victor Chernozhukov,
        Denis Chetverikov, Mert Demirer, Esther Duflo, Christian Hansen, and Whitney Newey. It is used to estimate
        the effect of the binary variable d on the outcome variable y, when X may be correlated with both d and y.

        Parameters
        ----------
        X: mxn numpy array where m is the number of observations and n is the number of regressors.
        y: numpy row vector of length m where y[i] corresponds to x[:,i]
        d: numpyrow vector of length m where d[i] corresponds to x[:,i]
        test_size : float, int (default=.5)
            If float, should be between 0.0 and 1.0 and represent the
            proportion of the dataset to include in the test split. If
            int, represents the absolute number of test samples. If None,
            the value is automatically set to the complement of the train size.
            If train size is also None, test size is set to 0.25.
        normalize: boolean, optional (default=True).
            If set to true, each regressor is normalized to have a standard deviation of 1 across the sample.
            This is strongly recommended for both lasso and ridge methods
        second_order_terms: boolean, optional (default=False)
            If set to true, then the machine learning method uses both all of the regressors included in X,
            and their second order terms (each regressor squared and interactive effects).
        verbose: boolean, optional (default=True).
            If set to true, then the beta and standard error results will be printed 
        standard_errors: string, optional (default="White")
            Options:
                -"Normal": results in normal standard errors
                -"White": results in heteroskedasticity robust standard errors
                    as in White 1980
                -"Mackinnon": results in alternative heteroskedasticity robust standard errors
                    as in Mackinnnon and White 1985
        """
        start=time.time()
        if np.sum(abs(d**2-d))>0:
            raise NameError("The row vector 'd' can only have values of 0 or 1")
        if min(len(X),len(y),len(d))<max(len(X),len(y),len(d)):
            raise NameError("X, y,and d all must have the same length")
        if second_order_terms:
            X=self._so_terms(X)
        if normalize:
            X=self._normalize_matrix(X)
        y_col=np.array([y]).T
        d_col=np.array([d]).T
        data=np.concatenate((y_col,d_col,X),axis=1)
        split=sklearn.model_selection.train_test_split(data,test_size=test_size)
        data_use=split[0]
        y_use=data_use[:,0]
        d_use=data_use[:,1]
        x_use=data_use[:,2:]
        data_out=split[1]
        y_out=data_out[:,0]
        d_out=data_out[:,1]
        x_out=data_out[:,2:]
        exp_d_x,res_d_x=self._find_residuals(d_use,d_out,x_use,x_out,binary_outcome=True)
        exp_y_x,res_y_x=self._find_residuals(y_use,y_out,x_use,x_out,binary_outcome=False)
        ols_model=sm.regression.linear_model.OLS(res_y_x,res_d_x)
        reg=ols_model.fit()
        res=sm.regression.linear_model.RegressionResults(ols_model, reg.params, normalized_cov_params=None, scale=1.0, cov_type='nonrobust', cov_kwds=None, use_t=None)
        self.pl_beta=reg.params[0]
        if standard_errors=="Mackinnon":
            self.pl_se=res.HC1_se[0]
        elif standard_errors=="Normal":
            self.pl_se=res.bse[0]
        elif standard_errors=="White":
            self.pl_se=res.HC0_se[0]
        else:
            raise NameError("You have entered an unrecognized standard errors parameter value.\nRecognized values include: 'Mackinnon', Normal', and 'White' ")	
        if verbose:
            print("Partial Linear Estimate Results")
            print("----------------------------")
            print("Continous outcome machine learning method:", self.method)
            print ("Binary outcome machine learning method:", self.method_binary)
            print("Beta=", self.pl_beta)
            print("SE=", self.pl_se)
            print("Completed in", time.time()-start, "seconds")
        return self
    def interactive_estimate(self,X,y,d,test_size=.5,normalize=True, second_order_terms=False, drop_zero_divide=False, modify_zero_divide=1E-3,verbose=True):
        """
        The interactive_estimate method is an implementation of the interactive estimation procedure developed 
        in `Double Machine  Learning for Treatment and Causal Parameters' by Victor Chernozhukov,
        Denis Chetverikov, Mert Demirer, Esther Duflo, Christian Hansen, and Whitney Newey. It is used to estimate
        the effect of the binary variable d on the outcome variable y, when X may be correlated with both d and y.

        Parameters
        ----------
        X: mxn numpy array where m is the number of observations and n is the number of regressors.
        y: numpy row vector of length m where y[i] corresponds to x[:,i]
        d: numpyrow vector of length m where d[i] corresponds to x[:,i]
        test_size : float, int (default=.5)
            If float, should be between 0.0 and 1.0 and represent the
            proportion of the dataset to include in the test split. If
            int, represents the absolute number of test samples. If None,
            the value is automatically set to the complement of the train size.
            If train size is also None, test size is set to 0.25.
        normalize: boolean, optional (default=True).
            If set to true, each regressor is normalized to have a standard deviation of 1 across the sample.
            This is strongly recommended for both lasso and ridge methods
        second_order_terms: boolean, optional (default=False)
            If set to true, then the machine learning method uses both all of the regressors included in X,
            and their second order terms (each regressor squared and interactive effects).
        drop_zero_divide: boolean, optional (default=False). 
            If the actual value of d_out[i] is 1 but the predicted value of dhat[i] is 0 (or visa versa),
            then the interactive estimate will necessarily have a divide by zero error.If drop_zero_divide
            is True, then all cases in which a divide by zero error would occur will be thrown out of the sample
        verbose: boolean, optional (default=True).
            If set to true, then the beta and standard error results will be printed 
        modify_zero_divide: float, optional (default=1E-3). modify_zero_divide is only used if drop_zero_divide
            is False. Whenever there is d_out[i]=1 and dhat[i]=0, dhat[i] is set to the value of modify_zero_divide.
            Similarly, whenever d_out[i]=0 and dhat[i]=1, then dhat[i] is set to the value of modify_zero_divide.
        """
        start=time.time()
        if np.sum(abs(d**2-d))>0:
            raise NameError("The row vector 'd' can only have values of 0 or 1")
        if min(len(X),len(y),len(d))<max(len(X),len(y),len(d)):
            raise NameError("X, y,and d all must have the same length")
        if second_order_terms:
            X=self._so_terms(X)
        if normalize:
            X=self._normalize_matrix(X)
        y_col=np.array([y]).T
        d_col=np.array([d]).T
        data=np.concatenate((y_col,d_col,X),axis=1)
        split=sklearn.model_selection.train_test_split(data,test_size=test_size)
        data_use=split[0]
        y_use=data_use[:,0]
        d_use=data_use[:,1]
        x_use=data_use[:,2:]
        index_use1=((d_use==1).T)
        index_use0=(0==index_use1)
        data_use1=data_use[index_use1]
        y_use1=y_use[index_use1]
        x_use1=x_use[index_use1]
        y_use0=y_use[index_use0]
        x_use0=x_use[index_use0]
        data_out=split[1]
        y_out=data_out[:,0]
        d_out=data_out[:,1]
        x_out=data_out[:,2:]
        yhat_d1,resy_d1=self._find_residuals(y_use1,y_out,x_use1,x_out,binary_outcome=False)
        yhat_d0,resy_d0=self._find_residuals(y_use0,y_out,x_use0,x_out,binary_outcome=False)
        dhat,resd=self._find_residuals(d_use,d_out,x_use,x_out,binary_outcome=True)
        phi=yhat_d1-yhat_d0+d_out*resy_d1/dhat-((1-d_out)*resy_d0/(1-dhat))
        dhat_zero=np.nonzero(1*(dhat==0))[0]
        dhat_one=np.nonzero(1*(dhat==1))[0]
        if drop_zero_divide:
            bad_val_list=[]
            for i in dhat_zero:
                if d_out[i]==1:
                    bad_val_list.append(i)	
                phi[i]=yhat_d1[i]-yhat_d0[i]-((1-d_out[i])*resy_d0[i]/(1-dhat[i]))
            for i in dhat_one:
                if d_out[i]==0:
                    bad_val_list.append(i)
                phi[i]=yhat_d1[i]-yhat_d0[i]+d_out[i]*resy_d1[i]/dhat[i]
            phi=np.delete(phi,bad_val_list)
            self.interactive_beta=np.mean(phi)
            self.interactive_se=np.std(phi)/math.sqrt(len(phi))
            if verbose:
                print("Interactive Estimate Results")
                print("----------------------------")
                print("Continous outcome machine learning method:", self.method)
                print ("Binary outcome machine learning method:", self.method_binary)
                print("Beta=", self.interactive_beta)
                print("SE=", self.interactive_se)
                print("Completed in", time.time()-start, "seconds")
            return self
        else:
            for i in dhat_zero:
                if d_out[i]==1:
                    d_out[i]=1-modify_zero_divide
                phi[i]=yhat_d1[i]-yhat_d0[i]-((1-d_out[i])*resy_d0[i]/(1-dhat[i]))
            for i in dhat_one:
                if d_out[i]==0:
                    d_out[i]=modify_zero_divide
                phi[i]=yhat_d1[i]-yhat_d0[i]+d_out[i]*resy_d1[i]/dhat[i]
            self.interactive_beta=np.mean(phi)
            self.interactive_se=np.std(phi)/math.sqrt(len(phi))
            if verbose:
                print("Interactive Estimate Results")
                print("----------------------------")
                print("Continous outcome machine learning method:", self.method)
                print ("Binary outcome machine learning method:", self.method_binary)
                print("Beta=", self.interactive_beta)
                print("SE=", self.interactive_se)
                print("Completed in", time.time()-start, "seconds")
            return self
    def _so_terms(self,X):
        """
        The function 'so_terms' creates a matrix featuring the first and second order terms for
        the input matrix X, where each row in X is an observation
        """
        X_copy=np.copy(X)
        dim1=len(X_copy[0])
        total_count=int((dim1**2)/2+dim1/2)
        x_squared=np.zeros((len(X),total_count))
        zero_list=[]
        for i in range(dim1):
            for j in range(i,dim1):
                dim2=dim1-i
                index=total_count-int((dim2**2)/2+dim2/2)+j-i
                x_squared[:,index]=X_copy[:,i]*X_copy[:,j]
                if i==j:
                    if np.sum(X_copy[:,i]-X_copy[:,i]**2)==0:
                        zero_list.append(index)
        x_squared_mod=np.delete(x_squared,zero_list,1)
        return np.concatenate((X_copy,x_squared_mod),axis=1)
    def _normalize_matrix(self,X):
        """
        The function 'normalize_matrix()' normalizes the input Matrix X by dividing each column in X
        by its standard deviation. X is an mxn numpy array where rows correspond to observations and 
        columns correspond to regressors
        """
        X_copy=np.copy(X)
        for i in range(len(X_copy[0])):
            X_copy[:,i:i+1]=X_copy[:,i:i+1]/np.std(X_copy[:,i:i+1])
        return X_copy

In [11]:
ML2Estimator(method="Random Forest").pl_estimate(x_var,y_var,d_var,second_order_terms=True,verbose=True)

Partial Linear Estimate Results
----------------------------
Continous outcome machine learning method: Random Forest
Binary outcome machine learning method: Random Forest
Beta= 9368.358535158572
SE= 1661.7301370742075
Completed in 83.93489098548889 seconds


In [12]:
ML2Estimator(method="Random Forest").interactive_estimate(x_var,y_var,d_var,second_order_terms=True,verbose=True)

Interactive Estimate Results
----------------------------
Continous outcome machine learning method: Random Forest
Binary outcome machine learning method: Random Forest
Beta= 4464.155620200678
SE= 2649.8602557056047
Completed in 84.7635440826416 seconds


In [13]:
ML2Estimator(method="Random Forest").pl_estimate(x_var,y_var,d_var,second_order_terms=False,verbose=True)

Partial Linear Estimate Results
----------------------------
Continous outcome machine learning method: Random Forest
Binary outcome machine learning method: Random Forest
Beta= 7673.116880833988
SE= 1617.6268368016645
Completed in 14.913313150405884 seconds


In [14]:
ML2Estimator(method="Random Forest").interactive_estimate(x_var,y_var,d_var,second_order_terms=False,verbose=True)

Interactive Estimate Results
----------------------------
Continous outcome machine learning method: Random Forest
Binary outcome machine learning method: Random Forest
Beta= 6757.202929663539
SE= 2203.8657526533575
Completed in 14.856983184814453 seconds
